In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TextSplitter

doc=TextLoader("D:\RAG\ml_ai_corpus.txt",encoding="utf-8")

docs=doc.load()


<>:4: SyntaxWarning: invalid escape sequence '\R'
<>:4: SyntaxWarning: invalid escape sequence '\R'
C:\Users\PARTH\AppData\Local\Temp\ipykernel_1656\1421567176.py:4: SyntaxWarning: invalid escape sequence '\R'
  doc=TextLoader("D:\RAG\ml_ai_corpus.txt",encoding="utf-8")


In [2]:

text_string=docs[0].page_content
text_string

'Machine Learning & Artificial Intelligence - Core Concepts and Overview\n\nSection 1 — Overview\nMachine Learning (ML) is a subfield of Artificial Intelligence (AI) that focuses on developing algorithms and statistical models that allow computers to perform tasks without explicit instructions. Instead, ML systems learn patterns from data and use those patterns to make predictions or decisions. AI is a broader discipline that includes ML as well as symbolic reasoning, planning, robotics, and other approaches to create systems that can simulate aspects of human intelligence.\n\nSection 2 — Brief History\nEarly AI research (1950s–1970s) explored symbolic reasoning and rule-based systems. ML emerged more clearly as computational power and data availability increased; key milestones include the development of decision trees, support vector machines, and neural networks. The 2010s brought deep learning advances, enabled by larger datasets and GPUs, making breakthroughs in computer vision, n

In [3]:
print(f"page conntent {docs[0].page_content[:800]}  ")
print(f"matadat metadata {docs[0].metadata}")

page conntent Machine Learning & Artificial Intelligence - Core Concepts and Overview

Section 1 — Overview
Machine Learning (ML) is a subfield of Artificial Intelligence (AI) that focuses on developing algorithms and statistical models that allow computers to perform tasks without explicit instructions. Instead, ML systems learn patterns from data and use those patterns to make predictions or decisions. AI is a broader discipline that includes ML as well as symbolic reasoning, planning, robotics, and other approaches to create systems that can simulate aspects of human intelligence.

Section 2 — Brief History
Early AI research (1950s–1970s) explored symbolic reasoning and rule-based systems. ML emerged more clearly as computational power and data availability increased; key milestones include the devel  
matadat metadata {'source': 'D:\\RAG\\ml_ai_corpus.txt'}


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk=RecursiveCharacterTextSplitter(
    separators=[",",".","/n","/n/n"],
    chunk_size=500,
    chunk_overlap=20
)
chunks=chunk.split_text(text_string)
print(f"lenght {len(chunks)}")


lenght 20


In [5]:
from langchain_openai import OpenAIEmbeddings
import os 

embed=OpenAIEmbeddings(
    api_key=os.getenv("api_key"),
    model="text-embedding-3-small",
    base_url=os.getenv("endpoints")
)

vector=embed.embed_documents(chunks)



d:\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import Chroma

vector = Chroma.from_texts(
    texts=chunks,
    embedding=embed,
    persist_directory="pipeline",
    collection_name="Queryenhancement"
)

retriever = vector.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5}
)


In [7]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001EFAB1DD400>, search_type='mmr', search_kwargs={'k': 5})

In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model="gpt-5-nano-2025-08-07",
    api_key=os.getenv("api_key"),
    openai_api_base=os.getenv("endpoints")
)

In [9]:
llm


ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EFADC28AD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EFADC28EC0>, root_client=<openai.OpenAI object at 0x000001EFADBBEE90>, root_async_client=<openai.AsyncOpenAI object at 0x000001EFADBBF4D0>, model_name='gpt-5-nano-2025-08-07', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.euron.one/api/v1/euri')

In [10]:
from langchain.prompts  import PromptTemplate
from langchain_core.output_parsers import StrOutputParser



prompt_llm=PromptTemplate.from_template(
    '''You are an AI assistant that enhances user queries for better information retrieval.

Your task is to take the original user query and generate an improved, expanded version of the query without changing its meaning.

Follow these rules:
- Keep the original intent exactly the same.
- Add relevant keywords, synonyms, and related phrases that might help retrieval.
- Include domain-specific terminology if applicable.
- Be concise and avoid unnecessary words.
- Do NOT answer the query. Only rewrite and enhance it.
- Return only the enhanced query. No explanations.

Original Query: {query}

Enhanced Query:
'''
)

query_expansion_chain=prompt_llm | llm |StrOutputParser()


In [11]:
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='You are an AI assistant that enhances user queries for better information retrieval.\n\nYour task is to take the original user query and generate an improved, expanded version of the query without changing its meaning.\n\nFollow these rules:\n- Keep the original intent exactly the same.\n- Add relevant keywords, synonyms, and related phrases that might help retrieval.\n- Include domain-specific terminology if applicable.\n- Be concise and avoid unnecessary words.\n- Do NOT answer the query. Only rewrite and enhance it.\n- Return only the enhanced query. No explanations.\n\nOriginal Query: {query}\n\nEnhanced Query:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EFADC28AD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EFADC28EC0>, root_client=<openai.OpenAI object at 0x000001EFADBBEE90>, root_as

In [13]:
query_expansion_chain.invoke({"query":"langchain"})

'LangChain framework for LLM-powered applications; LangChain Python and LangChain.js; chains, agents, prompts/templates, tools, memory; vector stores and embeddings (FAISS, Pinecone, Chroma); tool integrations; tutorials and documentation.'

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_template=ChatPromptTemplate.from_template("""
answer the question based on abbove context below 
 context :{context}
question :{input}""")

document_chain=create_stuff_documents_chain(llm=llm,prompt=prompt_template)

In [17]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough

rag_pipeline = (
    RunnableMap({
        "input": RunnablePassthrough(),
        "context": lambda x: retriever.invoke(
            query_expansion_chain.invoke(x["input"])
        )
    })
    | document_chain
)

In [27]:
query={"input":" what is AI and ML ?"}

In [28]:
answer=rag_pipeline.invoke(query)

In [31]:
answer

'- Artificial Intelligence (AI): A broad field that aims to create systems capable of simulating aspects of human intelligence. It includes ML and other approaches such as symbolic reasoning, planning, robotics, and more.\n\n- Machine Learning (ML): A subfield of AI that focuses on developing algorithms and statistical models that allow computers to perform tasks without explicit instructions. ML systems learn patterns from data and use those patterns to make predictions or decisions. Key ideas in ML include datasets, features, labels (targets), training, and validation.'